In [1]:
import pandas as pd
from itertools import cycle
path_blast = 'data/results/'
tes_type = ['MITE', 'TRIM', 'LARD', 'TIR','helitron','LTR','SINE', 'LINE']
pool = cycle(tes_type)

In [9]:
te_type = next(pool)
print('Running for', te_type)

('Running for', 'LINE')


In [31]:
#TEs
params = {}
params['LTR'] = {'min_len':650,'max_len':False,'min_distance':5,'max_q':1.5,'min_q':0.5,'min_pident':80,'min_qcov':40,'file':'blast_clustered_ltr.csv'}
params['LINE'] = {'min_len':1500,'max_len':False,'min_distance':5,'max_q':1.2,'min_q':0.8,'min_pident':80,'min_qcov':80,'file':'blast_clustered_line.csv'}
params['SINE'] = {'min_len':150,'max_len':800,'min_distance':5,'max_q':1.1,'min_q':0.9,'min_pident':90,'min_qcov':90,'file':'blast_clustered_sine.csv'}
params['TIR'] = {'min_len':500,'max_len':False,'min_distance':5,'max_q':1.1,'min_q':0.9,'min_pident':90,'min_qcov':90,'file':'blast_clustered_tir.csv'}
params['MITE'] = {'min_len':50,'max_len':800,'min_distance':5,'max_q':1.15,'min_q':0.85,'min_pident':90,'min_qcov':90,'file':'blast_clustered_mite.csv'}
params['LARD'] = {'min_len':50,'max_len':800,'min_distance':5,'max_q':1.15,'min_q':0.85,'min_pident':90,'min_qcov':90,'file':'blast_clustered_lard.csv'}
params['TRIM'] = {'min_len':2000,'max_len':False,'min_distance':5,'max_q':1.2,'min_q':0.8,'min_pident':80,'min_qcov':80,'file':'blast_clustered_trim.csv'}
params['helitron'] = {'min_len':2000,'max_len':False,'min_distance':5,'max_q':1.2,'min_q':0.8,'min_pident':80,'min_qcov':80,'file':'blast_clustered_helitron.csv'}
#select which TE type you want to run

In [32]:
#read blast output
df = pd.read_csv(path_blast + params[te_type]['file'], sep='\t', header=None)
df.columns = ['qseqid','sseqid','qstart','qend','sstart','send','score','length','mismatch','gaps','gapopen','nident','pident','qlen','slen','qcovs']
print('initial:',len(df.index))

('initial:', 1002)


In [33]:
#filter by length
if(params[te_type]['min_len']):
    df = df[(df.qlen > params[te_type]['min_len'])]
    
if(params[te_type]['max_len']):
    df = df[(df.qlen < params[te_type]['max_len'])]

print('Min and max len: ' + str(len(df.index)))

Min and max len: 697


In [35]:
df.head()

,qseqid,sseqid,qstart,qend,sstart,send,score,length,mismatch,gaps,gapopen,nident,pident,qlen,slen,qcovs
31,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr02,1,1780,19149564,19147794,3314,1785,26,19,10,1740,97.48,1780,48614681,100
32,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr02,1,1780,45325795,45327574,3302,1787,33,14,10,1740,97.37,1780,48614681,100
33,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr02,2,1090,4184680,4183624,1726,1092,54,38,7,1000,91.58,1780,48614681,100
34,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr03,1,1780,42680126,42678340,3254,1788,49,9,8,1730,96.76,1780,62290286,100
35,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr03,1,1780,23235851,23237633,2306,1786,238,9,8,1539,86.17,1780,62290286,100


In [36]:
#filter by query / subject length treshold
df = df[((df.length / df.qlen) >= params[te_type]['min_q'])]
print('min treshold:',len(df.index))

('min treshold:', 262)


In [37]:
df.head()

,qseqid,sseqid,qstart,qend,sstart,send,score,length,mismatch,gaps,gapopen,nident,pident,qlen,slen,qcovs
31,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr02,1,1780,19149564,19147794,3314,1785,26,19,10,1740,97.48,1780,48614681,100
32,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr02,1,1780,45325795,45327574,3302,1787,33,14,10,1740,97.37,1780,48614681,100
34,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr03,1,1780,42680126,42678340,3254,1788,49,9,8,1730,96.76,1780,62290286,100
35,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr03,1,1780,23235851,23237633,2306,1786,238,9,8,1539,86.17,1780,62290286,100
36,LINE_id_2_RIL_LINE32_FAM003189[type:LINE_sourc...,chr03,3,1780,36395558,36397321,2122,1791,242,40,18,1509,84.25,1780,62290286,100


In [30]:
df = ((df.length / df.qlen) <= params[te_type]['max_q'])
print('max treshold:',len(df.index))

('max treshold:', 262)


In [14]:
#filter by pident
df = df[(df.pident >= params[te_type]['min_pident'])]
print('Min_pident: ' + str(len(df.index)))

Min_pident: 262


In [15]:
#filter by qcov
df = df[(df.qcovs >= params[te_type]['min_qcov'])]
print('Min qcov: ' + str(len(df.index)))

Min qcov: 261


In [16]:
#order sstart and send
df['new_sstart'] = df[['sstart','send']].min(axis=1)
df['new_ssend'] = df[['sstart','send']].max(axis=1)
df['sstart'] = df['new_sstart']
df['send'] = df['new_ssend']
df = df.drop('new_sstart',axis=1).drop('new_ssend',axis=1)

# sep by chr
dfs = {}
for seq in df.sseqid.unique():
    dfs[seq] = df[df.sseqid == seq]


In [17]:
# filter overlapped 
rows = []
discard = []
total = len(df.index)
count = 0
curr = 0
for index, row in df.iterrows():
    count += 1
    curr_new = int(count * 100 * 1.0 / (total * 1.0))
    if curr_new != curr:
        curr = curr_new
        if curr_new % 10 == 0:
            print(curr_new)
    if index in discard:
        continue
    df_2 = dfs[row.sseqid]
    res = df_2[(abs(df_2.sstart - row.sstart) <= params[te_type]['min_distance']) | (abs(df_2.send - row.send) <= params[te_type]['min_distance'])]
    if len(res.index) > 1:
        discard.extend(res.index.values)
    rows.append(row)

10
20
30
40
50
60
70
80
90
100


In [18]:
df = pd.DataFrame(rows)
df.sort_values(['sseqid', 'sstart'], inplace=True)
print('Non overlapped: ' + str(len(df.index)))


Non overlapped: 248


In [19]:
filename = path_blast + params[te_type]['file'] + '.filtered'
df.to_csv(filename, index=None, sep='\t')
filename

'data/results/blast_clustered_line.csv.filtered'